In [565]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_colwidth', None)

In [3]:

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Step 1: Scrape listing links
i = 4290
url = f'https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/cala-polska?page={i}'
houselinks = []
while True:  # Adjust the range as needed
    i += 1
    r = requests.get(url, headers=headers)

    url = f'https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/cala-polska?page={i}'
    print(f'Page {i} status code:', r.status_code)
    soup = BeautifulSoup(r.content, 'lxml')

    # Check if no listings found
    h3_element = soup.find('h3', class_='css-1nw1os0 e1ws6l2x3')
    if h3_element and h3_element.get_text(strip=True) == 'Nie znaleźliśmy żadnych ogłoszeń':
        print("No advertisements found. Stopping the scraping process.")
        break

    listing_container = soup.find_all('section', class_='eeungyz1 css-hqx1d9 e12fn6ie0')
    if not listing_container:
        print(f"No listing container found on page {i}")
    else:
        print(f"Found {len(listing_container)} listing containers on page {i}")

    # Extract individual listing URLs
    for section in listing_container:
        links = section.find_all('a', class_='css-16vl3c1 e17g0c820')
        for link in links:
            href = link['href']
            if href and href not in houselinks:
                # Ensure the URL is complete
                if not href.startswith('http'):
                    href = 'https://www.otodom.pl' + href
                houselinks.append(href)

print(f"Total houselinks found: {len(houselinks)}")

# Step 2: Scrape details from each listing
houselist = []
for link in houselinks:
    r = requests.get(link, headers=headers)
    if r.status_code == 403:
        print(f"Access denied for link: {link}")
        continue

    soup = BeautifulSoup(r.text, 'lxml')
    body = soup.select_one('main')
    if not body:
        print(f"No main section found for {link}")
        continue

    # Check if no listings found
    h3_element = body.find('h3', class_='css-1nw1os0 e1ysrxc23')
    if h3_element and h3_element.get_text(strip=True) == 'Nie znaleźliśmy żadnych ogłoszeń':
        print("No advertisements found on this detail page. Stopping the scraping process.")
        break

    try:
        NumerId = link

        nazwa = body.find('h1', class_='css-1wnihf5 e1j8g12x8')
        nazwa = nazwa.text.strip() if nazwa else "N/A"

        cenaM2 = body.find('div', class_='css-1h1l5lm e1j8g12x9')
        cenaM2 = cenaM2.text.strip() if cenaM2 else "N/A"

        cena = body.find('strong', class_='css-t3wmkv e9aa0kv0')
        cena = cena.text.strip() if cena else "N/A"

        adres = body.find('a', class_='eozeyij0 css-1helwne e1p0dzoz0')
        adres = adres.text.strip() if adres else "N/A"

        powierzchnia = body.find('div', {'aria-label': 'Powierzchnia', 'class': 'css-1ivc1bc e26jmad1'})
        powierzchnia = powierzchnia.text.strip() if powierzchnia else "N/A"

        formaWlasnosci = body.find('div', {'aria-label': 'Forma własności', 'class': 'css-1ivc1bc e26jmad1'})
        formaWlasnosci = formaWlasnosci.text.strip() if formaWlasnosci else "N/A"

        LiczbaPokoi = body.find('div', {'aria-label': 'Liczba pokoi', 'class': 'css-1ivc1bc e26jmad1'})
        LiczbaPokoi = LiczbaPokoi.text.strip() if LiczbaPokoi else "N/A"

        stanWykonczenia = body.find('div', {'aria-label': 'Stan wykończenia','class':'css-1ivc1bc e26jmad1'})
        stanWykonczenia = stanWykonczenia.text.strip() if stanWykonczenia else "N/A"

        pietro = body.find('div', {'aria-label': 'Piętro', 'class': 'css-1ivc1bc e26jmad1'})
        pietro = pietro.text.strip() if pietro else "N/A"

        BalkonOgrodTaras = body.find('div', {'aria-label': 'Balkon / ogród / taras', 'class': 'css-1ivc1bc e26jmad1'})
        BalkonOgrodTaras = BalkonOgrodTaras.text.strip() if BalkonOgrodTaras else "N/A"

        Czynsz = body.find('div', {'aria-label': 'Czynsz', 'class': 'css-1ivc1bc e26jmad1'})
        Czynsz = Czynsz.text.strip() if Czynsz else "N/A"
        
        MiejsceParkingowe = body.find('div', {'aria-label': 'Miejsce parkingowe', 'class': 'css-1ivc1bc e26jmad1'})
        MiejsceParkingowe = MiejsceParkingowe.text.strip() if MiejsceParkingowe else "N/A"
        
        ObslugaZdalna = body.find('div', {'aria-label': 'Obsługa zdalna', 'class': 'css-1ivc1bc e26jmad1'})
        ObslugaZdalna = ObslugaZdalna.text.strip() if ObslugaZdalna else "N/A"

        ogrzewanie = body.find('div', {'aria-label': 'Ogrzewanie', 'class': 'css-1wi2w6s e26jmad5'})
        ogrzewanie = ogrzewanie.text.strip() if ogrzewanie else "N/A"

        Rynek = body.find('div', {'aria-label': 'Rynek', 'class': 'css-tpkder e26jmad1'})
        Rynek = Rynek.text.strip() if Rynek else "N/A"

        TypOgloszeniodawcy = body.find('div', {'aria-label': 'Typ ogłoszeniodawcy', 'class': 'css-tpkder e26jmad1'})
        TypOgloszeniodawcy = TypOgloszeniodawcy.text.strip() if TypOgloszeniodawcy else "N/A"

        DostepneOd = body.find('div', {'aria-label': 'Dostępne od', 'class': 'css-tpkder e26jmad1'})
        DostepneOd = DostepneOd.text.strip() if DostepneOd else "N/A"

        RokBudowy = body.find('div', {'aria-label': 'Rok budowy', 'class': 'css-tpkder e26jmad1'})
        RokBudowy = RokBudowy.text.strip() if RokBudowy else "N/A"

        RodzajZabudowy = body.find('div', {'aria-label': 'Rodzaj zabudowy', 'class': 'css-tpkder e26jmad1'})
        RodzajZabudowy = RodzajZabudowy.text.strip() if RodzajZabudowy else "N/A"
        
        Okna = body.find('div', {'aria-label': 'Okna', 'class': 'css-tpkder e26jmad1'})
        Okna = Okna.text.strip() if Okna else "N/A"

        CzyWinda = body.find('div', {'aria-label': 'Winda', 'class': 'css-tpkder e26jmad1'})
        CzyWinda = CzyWinda.text.strip() if CzyWinda else "N/A"
        
        Media = body.find('div', {'aria-label': 'Media', 'class': 'css-tpkder e26jmad1'})
        Media = Media.text.strip() if Media else "N/A"

        Zabezpieczenia = body.find('div', {'aria-label': 'Zabezpieczenia', 'class': 'css-tpkder e26jmad1'})
        Zabezpieczenia = Zabezpieczenia.text.strip() if Zabezpieczenia else "N/A"

        Wyposazenie = body.find('div', {'aria-label': 'Wyposażenie', 'class': 'css-tpkder e26jmad1'})
        Wyposazenie = Wyposazenie.text.strip() if Wyposazenie else "N/A"

        InformacjeDodatkowe = body.find('div', {'aria-label': 'Informacje dodatkowe', 'class': 'css-tpkder e26jmad1'})
        InformacjeDodatkowe = InformacjeDodatkowe.text.strip() if InformacjeDodatkowe else "N/A"

        MaterialBudynku = body.find('div', {'aria-label': 'Materiał budynku', 'class': 'css-tpkder e26jmad1'})
        MaterialBudynku = MaterialBudynku.text.strip() if MaterialBudynku else "N/A"

        rooms = {
            'ID': NumerId,
            'Nazwa': nazwa,
            'CenaM2': cenaM2,
            'Cena': cena,
            'Adres': adres,
            'Powierzchnia': powierzchnia,
            'Forma_Wlasnosci': formaWlasnosci,
            'LiczbaPokoi': LiczbaPokoi,
            'StanWykonczenia': stanWykonczenia,
            'Pietro': pietro,
            'BalkonOgrodTaras': BalkonOgrodTaras,
            'Czynsz': Czynsz,
            'Ogrzewanie': ogrzewanie,
            'Rynek': Rynek,
            'TypOgloszeniodawcy': TypOgloszeniodawcy,
            'DostepneOd': DostepneOd,
            'RokBudowy': RokBudowy,
            'RodzajZabudowy': RodzajZabudowy,
            'CzyWinda': CzyWinda,
            'Zabezpieczenia': Zabezpieczenia,
            'Wyposazenie': Wyposazenie,
            'InformacjeDodatkowe': InformacjeDodatkowe,
            'MaterialBudynku': MaterialBudynku
        }
        houselist.append(rooms)
    except AttributeError as e:
        #print(f"Error parsing details for {link}: {e}")
        continue

# Save the data to a DataFrame
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(houselist)

Page 4291 status code: 200
Found 39 listing containers on page 4291
Page 4292 status code: 200
Found 39 listing containers on page 4292
Page 4293 status code: 200
Found 39 listing containers on page 4293
Page 4294 status code: 200
Found 5 listing containers on page 4294
Page 4295 status code: 200
No advertisements found. Stopping the scraping process.
Total houselinks found: 122


,ID,Nazwa,CenaM2,Cena,Adres,Powierzchnia,Forma_Wlasnosci,LiczbaPokoi,StanWykonczenia,Pietro,...,Rynek,TypOgloszeniodawcy,DostepneOd,RokBudowy,RodzajZabudowy,CzyWinda,Zabezpieczenia,Wyposazenie,InformacjeDodatkowe,MaterialBudynku
0,https://www.otodom.pl/pl/oferta/3-pokojowe-mieszkanie-72m2-loggia-bezposrednio-ID4pKVC,3-pokojowe mieszkanie 72m2 + loggia Bezpośrednio,,Zapytaj o cenę,"ul. Małokacka, Witomino-Radiostacja, Gdynia, pomorskie","Powierzchnia72,31 m²",Forma własnościpełna własność,Liczba pokoi3,Stan wykończeniado wykończenia,Piętroparter/10,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,https://www.otodom.pl/pl/oferta/wyjatkowy-penthouse-winda-wjezdza-do-mieszkania-ID4rcjy,Wyjątkowy penthouse - winda wjeżdza do mieszkania!,13 606 zł/m²,1 650 000 zł,"ul. Cybińska, Śródka, Nowe Miasto, Poznań, wielkopolskie","Powierzchnia121,27 m²",Forma własnościpełna własność,Liczba pokoi4,Stan wykończeniado zamieszkania,Piętro4/4,...,Rynekwtórny,Typ ogłoszeniodawcyprywatny,Dostępne odbrak informacji,Rok budowy2008,Rodzaj zabudowyblok,Windatak,"Zabezpieczeniasystem alarmowy, drzwi / okna antywłamaniowe, domofon / wideofon","Wyposażeniezmywarka, lodówka, meble, piekarnik, kuchenka, pralka","Informacje dodatkoweklimatyzacja, pom. użytkowe",Materiał budynkubrak informacji
2,https://www.otodom.pl/pl/oferta/3-pokojowe-blisko-szkola-przedszkole-sklepy-ID4oTcd,"3-pokojowe, blisko szkoła, przedszkole, sklepy",4 900 zł/m²,295 000 zł,"Kętrzyn, kętrzyński, warmińsko-mazurskie","Powierzchnia60,2 m²",Forma własnościspółdzielcze wł. prawo do lokalu,Liczba pokoi3,Stan wykończeniado zamieszkania,Piętro4/4,...,Rynekwtórny,Typ ogłoszeniodawcyprywatny,Dostępne odbrak informacji,Rok budowy1984,Rodzaj zabudowyblok,Windanie,Zabezpieczeniadomofon / wideofon,"Wyposażeniemeble, piekarnik, kuchenka","Informacje dodatkowepiwnica, oddzielna kuchnia",Materiał budynkuwielka płyta
3,https://www.otodom.pl/pl/oferta/apartament-z-ogrodkiem-wysoki-standard-2-ID45uz8,Apartament z ogródkiem | wysoki standard | 2,11 146 zł/m²,915 000 zł,"Warszewo, Północ, Szczecin, zachodniopomorskie","Powierzchnia82,09 m²",Forma własnościZapytaj,Liczba pokoi3,Stan wykończeniado zamieszkania,Piętroparter/3,...,Rynekpierwotny,Typ ogłoszeniodawcybiuro nieruchomości,Dostępne od2020-04-14,Rok budowy2020,Rodzaj zabudowyapartamentowiec,Windatak,"Zabezpieczeniasystem alarmowy, monitoring / ochrona, domofon / wideofon",Wyposażeniebrak informacji,Informacje dodatkowebrak informacji,Materiał budynkubrak informacji
4,https://www.otodom.pl/pl/oferta/mieszkanie-30-70-m-nowe-drozdowo-ID45r5r,"Mieszkanie, 30,70 m², Nowe Drozdowo",4 984 zł/m²,153 000 zł,"Drozdowo, Rzewnie, makowski, mazowieckie","Powierzchnia30,7 m²",Forma własnościpełna własność,Liczba pokoi1,Stan wykończeniado wykończenia,Piętroparter,...,Rynekpierwotny,Typ ogłoszeniodawcybiuro nieruchomości,Dostępne odbrak informacji,Rok budowy2020,Rodzaj zabudowybrak informacji,Windanie,Zabezpieczeniabrak informacji,Wyposażeniebrak informacji,Informacje dodatkowebrak informacji,Materiał budynkubrak informacji
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,https://www.otodom.pl/pl/oferta/kupmnie-3pok-siemaszki-najnizsza-cena-w-centrum-ID4q782,KupMnie.3pok Siemaszki.Najniższa cena w Centrum.,14 279 zł/m²,637 000 zł,"ul. ks. Kazimierza Siemaszki, Żabiniec, Prądnik Biały, Kraków, małopolskie","Powierzchnia44,61 m²",Forma własnościpełna własność,Liczba pokoi3,Stan wykończeniado zamieszkania,Piętro3/10,...,Rynekwtórny,Typ ogłoszeniodawcybiuro nieruchomości,Dostępne odbrak informacji,Rok budowy1966,Rodzaj zabudowyblok,Windatak,Zabezpieczeniabrak informacji,Wyposażeniebrak informacji,Informacje dodatkoweoddzielna kuchnia,Materiał budynkużelbet
118,https://www.otodom.pl/pl/oferta/wyjatkowy-penthouse-winda-wjezdza-do-mieszkania-ID4rcjy,Wyjątkowy penthouse - winda wjeżdza do mieszkania!,13 606 zł/m²,1 650 000 zł,"ul. Cybińska, Śródka, Nowe Miasto, Poznań, wielkopolskie","Powierzchnia121,27 m²",Forma własnościpełna własn

In [617]:
df_copy = df.copy()

In [569]:
## FEATURE ENGINEERING

In [570]:
## ID
df_copy['ID'] = df_copy['ID'].apply(lambda x: re.search(r'(ID\w+)', x).group(0) if re.search(r'(ID\w+)', x) else None)

In [571]:
# Cena
df_copy['Cena'] = df_copy['Cena'].apply(lambda y: str(y).split('zł')[0])
df_copy['Cena'] = df_copy['Cena'].apply(lambda x: x.replace(' ', ''))
df_copy['Cena'] = df_copy['Cena'].apply(lambda x: x.replace(',', '.'))
df_copy['Cena'] = df_copy['Cena'].apply(lambda x: x.replace('Zapytajocenę',''))
df_copy['Cena'] = df_copy['Cena'].replace(['N/A', ''], np.nan)
df_copy['Cena'] = df_copy['Cena'].replace('', np.nan)
df_copy['Cena'] = df_copy['Cena'].astype(float)


In [572]:
# CenaM2
df_copy['CenaM2'] = df_copy['CenaM2'].apply(lambda y: str(y).split('zł/m²')[0])
df_copy['CenaM2'] = df_copy['CenaM2'].apply(lambda x: x.replace(' ', ''))
df_copy['CenaM2'] = df_copy['CenaM2'].replace('', np.nan)
df_copy['CenaM2'] = df_copy['CenaM2'].replace(['N/A', ''], np.nan)
df_copy['CenaM2'] = df_copy['CenaM2'].astype(float)

In [573]:
## Nazwa czy potrzebna ?

In [574]:
## Adres
### Wyodrębnienie nazwy wojewodztwa

wojewodztwa = ['dolnośląskie','kujawsko-pomorskie','lubelskie','lubuskie','łódzkie',
               'małopolskie','mazowieckie','opolskie','podkarpackie','podlaskie','pomorskie',
               'śląskie','świętokrzyskie','warmińsko-mazurskie','wielkopolskie','zachodniopomorskie']



In [575]:
df_copy['Wojewodztwo'] = df_copy['Adres'].str.split().apply(lambda x: '_'.join([m for m in x if m in wojewodztwa])).replace('',np.nan)

In [576]:
### Wyodrębnienie nazwy miast
file_name = r'C:\Users\seba_\Desktop\PythonProject\OtoDom\ListOfCity\SIMC_15-06-2024 ({}).csv'
df_city_poland = pd.concat([pd.read_csv(file_name.format(i),sep=';') for i in range(0, 16)])

In [577]:
city_set = {city.strip().lower() for city in df_city_poland['NAZWA']}

def extract_city_values(address):
    parts = [part.strip().lower() for part in address.split(',')]
    city = next((part for part in parts if part in city_set), np.nan)
    return city.capitalize() if city is not np.nan else city

df_copy['Miasto'] = df_copy['Adres'].apply(extract_city_values)


In [578]:
### Wyodrębnienie nazwy powiatow

In [579]:
df_district_poland = pd.read_excel(r'C:\Users\seba_\Desktop\PythonProject\OtoDom\ListofDistrict\ListaPowiatow.xlsx')

In [580]:
district_set = {district.strip().lower() for district in df_district_poland['Powiat']}

def extract_district_values(address):
    parts = [part.strip().lower() for part in address.split(',')]
    district = next((part for part in parts if part in district_set), np.nan)
    return district.capitalize() if district is not np.nan else district

df_copy['Powiat'] = df_copy['Adres'].apply(extract_district_values)

In [581]:
### Wyodrębnienie nazwy ulicy

In [582]:
def extract_ulica_values(address):
    matches = re.findall(r'\bul\.\s*[^,]*', address)
    return ', '.join(matches) if matches else np.nan

df_copy['Ulica'] = df_copy['Adres'].apply(extract_ulica_values)

In [583]:
##Powierzchnia

df_copy['Powierzchnia'] = df_copy['Powierzchnia'].str.replace('Powierzchnia', '').str.strip()
df_copy['Powierzchnia'] = df_copy['Powierzchnia'].apply(lambda y: str(y).split('m²')[0])
df_copy['Powierzchnia'] = df_copy['Powierzchnia'].apply(lambda x: x.replace(',', '.'))
df_copy['Powierzchnia'] = df_copy['Powierzchnia'].replace(['N/A', ''], np.nan)
df_copy['Powierzchnia'] = df_copy['Powierzchnia'].astype(float)

In [584]:
##Forma_Wlasnosci

df_copy['Forma_Wlasnosci'] = df_copy['Forma_Wlasnosci'].str.replace('Forma własności', '').str.strip()

In [585]:
df_copy['LiczbaPokoi'] = df_copy['LiczbaPokoi'].str.replace('Liczba pokoi', '').str.strip()
df_copy['LiczbaPokoi'] = df_copy['LiczbaPokoi'].replace(['N/A', 'NaN', ''], np.nan)
df_copy['LiczbaPokoi'] = df_copy['LiczbaPokoi'].fillna(0).astype(int)

In [586]:
##StanWykonczenia
df_copy['StanWykonczenia'] = df_copy['StanWykonczenia'].str.replace('Stan wykończenia', '').str.strip()

In [587]:
##Pietro
df_copy['Pietro'] = df_copy['Pietro'].str.replace('Piętro', '').str.strip()
df_copy['Pietro'] = df_copy['Pietro'].str.replace('parter', '0')
df_copy['Pietro'] = df_copy['Pietro'].replace('poddasze', '100')
df_copy['Pietro'] = df_copy['Pietro'].replace('Zapytaj', '')
df_copy['Pietro'] = df_copy['Pietro'].replace('>', '', regex=True)
df_copy['Pietro'] = df_copy['Pietro'].replace('', np.nan)
df_copy['Pietro'] = df_copy['Pietro'].apply(lambda x: int(x.split('/')[0]) if isinstance(x, str) and '/' in x else x)
df_copy['Pietro'] = df_copy['Pietro'].astype(float)

In [588]:
##RodzajZabudowy
df_copy['RodzajZabudowy'] = df_copy['RodzajZabudowy'].str.replace('Rodzaj zabudowy', '').str.strip()

In [620]:
##CzyWinda
df_copy['CzyWinda'] = df_copy['CzyWinda'].str.replace('Winda', '').str.strip()
df_copy['CzyWinda'] = df_copy['CzyWinda'].replace(['N/A', 'NaN', ''], 'Brak_informacji')

In [590]:
##Zabezpieczenia
df_copy['Zabezpieczenia'] = df_copy['Zabezpieczenia'].str.replace('Zabezpieczenia', '').str.strip()

In [591]:
##Wyposazenie
df_copy['Wyposazenie'] = df_copy['Wyposazenie'].str.replace('Wyposażenie', '').str.strip()

In [592]:
##InformacjeDodatkowe
df_copy['InformacjeDodatkowe'] = df_copy['InformacjeDodatkowe'].str.replace('Informacje dodatkowe', '').str.strip()

In [593]:
##MaterialBudynku
df_copy['MaterialBudynku'] = df_copy['MaterialBudynku'].str.replace('Materiał budynku', '').str.strip()

In [594]:
df_copy['RokBudowy'] = df_copy['RokBudowy'].str.replace('Rok budowy', '').str.strip()
df_copy['RokBudowy'] = df_copy['RokBudowy'].replace(['N/A', ''], np.nan)
df_copy['RokBudowy'] = df_copy['RokBudowy'].replace(['brak informacji', ''], np.nan)
df_copy['RokBudowy'] = df_copy['RokBudowy'].astype(float)

In [595]:
df_copy['DostepneOd'] = df_copy['DostepneOd'].str.replace('Dostępne od', '').str.strip()
df_copy['DostepneOd'] = df_copy['DostepneOd'].replace(['N/A', ''], np.nan)
df_copy['DostepneOd'] = df_copy['DostepneOd'].replace(['brak informacji', ''], np.nan)

df_copy['DostepneOd'] = pd.to_datetime(df_copy['DostepneOd'])

In [596]:
df_copy['BalkonOgrodTaras'] = df_copy['BalkonOgrodTaras'].str.replace('Balkon / ogród / taras', '').str.strip()
df_copy['BalkonOgrodTaras'] = df_copy['BalkonOgrodTaras'].apply(lambda x: x.replace('[Zapytaj]',''))

In [597]:
df_copy['Czynsz'] = df_copy['Czynsz'].apply(lambda y: str(y).split('zł')[0])
df_copy['Czynsz'] = df_copy['Czynsz'].apply(lambda x: x.replace(' ', ''))
df_copy['Czynsz'] = df_copy['Czynsz'].apply(lambda x: x.replace(',', '.'))
df_copy['Czynsz'] = df_copy['Czynsz'].str.replace('Czynsz', '').str.strip()
df_copy['Czynsz'] = df_copy['Czynsz'].apply(lambda x: x.replace('Zapytaj',''))
df_copy['Czynsz'] = df_copy['Czynsz'].replace(['N/A', ''], np.nan)
df_copy['Czynsz'] = df_copy['Czynsz'].astype(float)

In [598]:
df_copy['Zabezpieczenia'] = df_copy['Zabezpieczenia'].replace('N/A', 'brak informacji')

In [599]:
df_copy['Zabezpieczenia'] = df_copy['Zabezpieczenia'].str.split(', ')
all_features = set(feature for sublist in df_copy['Zabezpieczenia'] for feature in sublist)

for feature in all_features:
    df_copy[feature] = df_copy['Zabezpieczenia'].apply(lambda x: 1 if feature in x else 0)
    

In [600]:
df_copy['Wyposazenie'] = df_copy['Wyposazenie'].replace('N/A', 'brak informacji')

In [601]:
df_copy['Wyposazenie'] = df_copy['Wyposazenie'].str.split(', ')
all_features = set(feature for sublist in df_copy['Wyposazenie'] for feature in sublist)

for feature in all_features:
    df_copy[feature] = df_copy['Wyposazenie'].apply(lambda x: 1 if feature in x else 0)

In [602]:
df_copy['BalkonOgrodTaras'] = df_copy['BalkonOgrodTaras'].str.split(', ')
all_features = set(feature for sublist in df_copy['BalkonOgrodTaras'] for feature in sublist)

for feature in all_features:
    df_copy[feature] = df_copy['BalkonOgrodTaras'].apply(lambda x: 1 if feature in x else 0)

In [603]:
df_copy.drop(['Nazwa','Adres','BalkonOgrodTaras','Zapytaj','DostepneOd','brak informacji'], axis = 1, inplace = True)

In [621]:
df_copy['CzyWinda']

0      Brak_informacji
1                  tak
2                  nie
3                  tak
4                  nie
            ...       
117                tak
118                tak
119                tak
120                nie
121                nie
Name: CzyWinda, Length: 122, dtype: object